In [21]:
import plaidml.keras
plaidml.keras.install_backend()
import keras
from sklearn.model_selection import train_test_split


import os 
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
labels = ['cracked', 'non-cracked']
img_size = 128

#https://www.analyticsvidhya.com/blog/2020/10/create-image-classification-model-python-keras/
def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)


data = get_data("Combined")

'NoneType' object is not subscriptable


C:\Users\Qthooloo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  app.launch_new_instance()


In [23]:
X = []
y = []

for feature, label in data:
    X.append(feature)
    y.append(label)

# Normalize
X = np.array(X) / 255

# Reshape
X.reshape(-1, img_size, img_size, 1)

# Convert to Numpy Array
y = np.array(y)

data = []

In [39]:
print("Count of Cracked - " + str(sum(y == 0)))
print("Count of Non-Cracked - " + str(sum(y == 1)))

print("Ratio of non-cracked to cracked ~ " + str(round((sum(y == 1) / (sum(y == 1) + sum(y == 0))),3)))


Count of Cracked - 8484
Count of Non-Cracked - 47608
Ratio of non-cracked to cracked ~ 0.849


In [4]:
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8, random_state=24)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5, random_state=24)

X = []
y = []

In [7]:
def generate_model():
    return keras.models.Sequential([
        keras.layers.Conv2D(32, (32, 32), input_shape=(128,128,3)),
        keras.layers.Activation('relu'),
        keras.layers.Conv2D(256, (3, 3), strides=(2,2)),
        keras.layers.Activation('relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Activation('relu'),
        keras.layers.Conv2D(256, (3, 3), strides=(2,2)),
        keras.layers.Activation('relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Activation('relu'),
        keras.layers.Flatten(),
        keras.layers.Dense(10),
        keras.layers.Activation('softmax')
    ])

model = generate_model()

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 97, 97, 32)        98336     
_________________________________________________________________
activation_7 (Activation)    (None, 97, 97, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 48, 48, 256)       73984     
_________________________________________________________________
activation_8 (Activation)    (None, 48, 48, 256)       0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 24, 24, 256)       0         
_________________________________________________________________
activation_9 (Activation)    (None, 24, 24, 256)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 11, 11, 256)       590080    
__________

In [8]:
opt = keras.optimizers.RMSprop(lr=0.001, decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=256, epochs=25, validation_data=(X_valid, y_valid), shuffle=True)

scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Train on 44873 samples, validate on 5609 samples
Epoch 1/25
44873/44873 [==============================] - 190s 4ms/step - loss: 0.6314 - acc: 0.8238 - val_loss: 0.4337 - val_acc: 0.8511
Epoch 2/25
44873/44873 [==============================] - 171s 4ms/step - loss: 0.4490 - acc: 0.8486 - val_loss: 0.4496 - val_acc: 0.8511
Epoch 3/25
44873/44873 [==============================] - 171s 4ms/step - loss: 0.4276 - acc: 0.8486 - val_loss: 0.4215 - val_acc: 0.8511
Epoch 4/25
 1024/44873 [..............................] - ETA: 2:51 - loss: 0.4247 - acc: 0.8496

KeyboardInterrupt: 